In [8]:
import torch
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer,TextClassificationPipeline

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [4]:
from datasets import load_dataset
raw_datasets=load_dataset('csv',data_files={'train': 'train.csv',
                                              'test':'test.csv'})

Using custom data configuration default-0465f68cbeafdcaf
Reusing dataset csv (C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 2/2 [00:00<00:00, 34.60it/s]


In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200000
    })
})

In [5]:
'''model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)'''

'model_name = "siebert/sentiment-roberta-large-english"\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name)\ntrainer = Trainer(model=model)'

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [9]:
tokenizer_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)
model = transformers.DistilBertForSequenceClassification.from_pretrained(model_name).to('cuda')
trainer = Trainer(model=model)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-14b885d58d73eded.arrow
Loading cached processed dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-c5a5541ea1f4db19.arrow


In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=10).select(range(2000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=10).select(range(200)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-69653c1aeeb3edb3.arrow
Loading cached shuffled indices for dataset at C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-0465f68cbeafdcaf\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-0fe216909a5212df.arrow


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate = 2e-5,
    #batch_size = 32,
    warmup_steps=0 ,
    #max_seq_length = 128,
    num_train_epochs = 5.0 ,
    weight_decay=0.01 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [34]:
torch.cuda.is_available()

True

In [35]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
***** Running training *****
  Num examples = 2000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
 40%|████      | 500/1250 [00:45<01:09, 10.80it/s]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 0.0528, 'learning_rate': 1.2e-05, 'epoch': 2.0}


Model weights saved in ./results\checkpoint-500\pytorch_model.bin
 80%|████████  | 1000/1250 [01:33<00:23, 10.47it/s]Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json


{'loss': 0.0214, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.0}


Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
100%|██████████| 1250/1250 [01:57<00:00, 10.70it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1250/1250 [01:57<00:00, 10.63it/s]

{'train_runtime': 117.6415, 'train_samples_per_second': 85.004, 'train_steps_per_second': 10.626, 'train_loss': 0.03214371929168701, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.03214371929168701, metrics={'train_runtime': 117.6415, 'train_samples_per_second': 85.004, 'train_steps_per_second': 10.626, 'train_loss': 0.03214371929168701, 'epoch': 5.0})

In [36]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
100%|██████████| 25/25 [00:00<00:00, 34.93it/s]


{'eval_loss': 1.5619434118270874,
 'eval_accuracy': 0.855,
 'eval_runtime': 1.2852,
 'eval_samples_per_second': 155.62,
 'eval_steps_per_second': 19.452}

In [39]:
trainer.save_model('C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english')

Saving model checkpoint to C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english
Configuration saved in C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english\config.json
Model weights saved in C:\RMM\Medical3\distilbert-base-uncased-finetuned-sst-2-english\pytorch_model.bin
